In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
adultlist = np.loadtxt('../data/adultpre.csv', delimiter = ',' , dtype = np.int32, skiprows=1)

In [3]:
adultlist.shape

(30162, 13)

In [4]:
adultlist

array([[    0,    39,    13, ...,     1,     0,     1],
       [    1,    50,    13, ...,     1,     2,     1],
       [    2,    38,     9, ...,     1,     1,     1],
       ...,
       [32558,    58,     9, ...,     1,     1,     1],
       [32559,    22,     9, ...,     1,     1,     1],
       [32560,    52,     9, ...,     1,     2,     0]])

In [5]:
x_data = adultlist[:, 0:-1]
y_data = adultlist[:, [-1]]

# feed_dict에 feed될(to be fed) tensor를 위한 placeholder 설정
X = tf.placeholder(tf.float32, shape=[None, 12])
Y = tf.placeholder(tf.float32, shape=[None, 1])

# tf.Variable(변수)에 random number를 각각 assign
# (21113, 1) = (105, 4) @  (4, 3)
W = tf.Variable(tf.random_normal([12, 1], mean=0.01, stddev=0.01), name='weight')
# W = tf.get_variable('W1', shape=[12,1], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([1]), name='bias')

# logistic regression Hypothesis 설정(tf.sigmoid() == 0과 1 사이의 값을 만들기 위한 시그모이드 함수)
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
hypothesis = tf.clip_by_value(hypothesis, 1e-5, 1-(1e-5))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
x_data.shape

(30162, 12)

In [7]:
y_data.shape

(30162, 1)

In [8]:
train_size = int(len(x_data) * 0.7)

In [9]:
train_size

21113

In [10]:
x_train, x_test = x_data[:train_size], x_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

In [11]:
x_train.shape

(21113, 12)

In [12]:
y_train.shape

(21113, 1)

In [13]:
x_train

array([[    0,    39,    13, ...,    40,     1,     0],
       [    1,    50,    13, ...,    13,     1,     2],
       [    2,    38,     9, ...,    40,     1,     1],
       ...,
       [22777,    28,     9, ...,    40,     1,     0],
       [22778,    27,    10, ...,    40,     1,     1],
       [22779,    27,     9, ...,     8,     1,     1]])

In [14]:
# 2. Cost function 최소화

#cost function(logistic regression에서, W와 b를 찾기 위한 cost)
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
# train =  tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
train =  tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

# 정확성 측정
# tf.cast()의 첫 번째 파라미터 'if hypothesis > 0.5' then 1'  1 을 float32로 캐스팅(=1.0)
predicted =  tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [15]:
# 3. 학습을 통해 실행

with tf.Session() as sess:     
    sess.run(tf.global_variables_initializer())
    feed = {X: x_train, Y: y_train}
    for step in range(34001):
        sess.run(train, feed_dict=feed)
        if step % 2000 == 0:
            print(step, sess.run(cost, feed_dict=feed))
       
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_test, Y: y_test})
    print("\nHypothesis: ", h,"\nCorrect (Y): ", c, "\nAccuracy: ", a)

0 2.804402
2000 0.59109
4000 0.53209406
6000 0.4924976
8000 0.46705145
10000 0.44986072
12000 0.43711036
14000 0.42699847
16000 0.41884896
18000 0.41230097
20000 0.40700793
22000 0.40270498
24000 0.39920968
26000 0.39638168
28000 0.39403436
30000 0.39194134
32000 0.38996625
34000 0.38807586

Hypothesis:  [[0.7148025 ]
 [0.9743889 ]
 [0.5198188 ]
 ...
 [0.9377319 ]
 [0.98609424]
 [0.34649462]] 
Correct (Y):  [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [0.]] 
Accuracy:  0.8108078


In [33]:
epochs = 30000
# epochs = 1

In [34]:
# Hypothesis 
hypothesis = tf.nn.sigmoid(tf.add(tf.matmul(X, W), b)) 
  
# Sigmoid Cross Entropy Cost Function 
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = hypothesis, labels = Y))
  
# Gradient Descent Optimizer 
# train = tf.train.GradientDescentOptimizer( learning_rate=0.0035).minimize(cost) 
train = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)
  
# Global Variables Initializer 
init = tf.global_variables_initializer() 

In [35]:
# with tf.Session() as sess:     
#     sess.run(tf.global_variables_initializer())
#     feed = {X: x_train, Y: y_train}
#     for step in range(34001):
#         sess.run(train, feed_dict=feed)
#         if step % 2000 == 0:
#             print(step, sess.run(cost, feed_dict=feed))
       
#     h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_test, Y: y_test})
#     print("\nHypothesis: ", h,"\nCorrect (Y): ", c, "\nAccuracy: ", a)
    

cost_history = np.empty(shape=[1],dtype=float)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        _, c = sess.run([train, cost], feed_dict={X: x_train, Y: y_train})
        
        if epoch % 5000 == 0 and epoch != 0: 
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c),"W=", sess.run(W), "b=", sess.run(b))
        cost_history = np.append(cost_history, c)
        
    # Calculate the correct predictions
    correct_prediction = tf.to_float(tf.greater(hypothesis, 0.5))

    # Calculate accuracy on the test set
    accuracy = tf.reduce_mean(tf.to_float(tf.equal(Y, correct_prediction)))

    print ("Train Accuracy:", accuracy.eval({X: x_train, Y: y_train}))
    print ("Test Accuracy:", accuracy.eval({X: x_test, Y: y_test}))

Epoch: 5001 cost= 0.559699416 W= [[ 0.02466656]
 [ 0.10063773]
 [ 0.08433592]
 [ 0.07065304]
 [ 0.07668696]
 [ 0.08329215]
 [ 0.10808203]
 [-0.00470729]
 [ 0.06450351]
 [ 0.09892242]
 [ 0.06618963]
 [ 0.08709431]] b= [-1.7165184]
Epoch: 10001 cost= 0.559698641 W= [[ 0.03104849]
 [ 0.13603379]
 [ 0.12706955]
 [ 0.08362817]
 [ 0.10308339]
 [ 0.1311587 ]
 [ 0.14870882]
 [-0.00452863]
 [ 0.0782023 ]
 [ 0.1292602 ]
 [ 0.10261149]
 [ 0.11465703]] b= [-1.6821399]
Epoch: 15001 cost= 0.559698582 W= [[ 0.0373275 ]
 [ 0.1714619 ]
 [ 0.17216085]
 [ 0.0939889 ]
 [ 0.12899804]
 [ 0.18406302]
 [ 0.18657552]
 [-0.00392068]
 [ 0.08171138]
 [ 0.15912037]
 [ 0.13986918]
 [ 0.13950892]] b= [-1.6472172]
Epoch: 20001 cost= 0.559698582 W= [[ 0.0439777 ]
 [ 0.20835505]
 [ 0.21868704]
 [ 0.10053502]
 [ 0.1524328 ]
 [ 0.22516043]
 [ 0.2093192 ]
 [-0.00375775]
 [ 0.08199652]
 [ 0.1899374 ]
 [ 0.16831692]
 [ 0.15684035]] b= [-1.6201214]
Epoch: 25001 cost= 0.559698582 W= [[ 0.05182057]
 [ 0.24452528]
 [ 0.2538519 